In [1]:
import pandas as pd
import plotly.graph_objects as go
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

d:\study\time_forecust\time-series-forecating\venv1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv("data.csv")
df = df.rename(columns={"дата": "Date", "направление": "direction", "выход": "value"})
df = df[["Date", "value"]]

df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df = df.iloc[::-1].reset_index(drop=True)
df["value"] = df["value"].str.replace(",", ".").astype(float)
df

In [ ]:
fig = go.Figure([go.Scatter(x=df["Date"], y=df["value"])])
fig.show()

In [ ]:
df = df.rename(columns={"Date": "ds", "value": "y"})

train = df.iloc[:]
test = df.iloc[len(df) - 238 :]

model = Prophet()
model.fit(train)

In [ ]:
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

iplot(
    [
        go.Scatter(x=df["ds"], y=df["y"], name="fact"),
        go.Scatter(x=forecast["ds"], y=forecast["yhat"], name="prediction"),
        go.Scatter(
            x=forecast["ds"],
            y=forecast["yhat_upper"],
            fill="tonexty",
            mode="none",
            name="upper",
        ),
        go.Scatter(
            x=forecast["ds"],
            y=forecast["yhat_lower"],
            fill="tonexty",
            mode="none",
            name="lower",
        ),
        go.Scatter(x=forecast["ds"], y=forecast["trend"], name="trend"),
    ]
)

In [ ]:
df

In [ ]:
forecast["yhat"]

In [ ]:
plot_plotly(model, forecast)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [44]:
data_column = "direction"
df = pd.read_csv("data.csv")
df = df.rename(columns={"дата": "Date", "направление": "direction", "выход": "value"})
df = df[["Date", data_column]]

df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df = df.iloc[::-1].reset_index(drop=True)

df = df.replace({"direction": {"ш": 0, "л": 1}})
df[data_column] = df[data_column].astype(float)

df

C:\Users\laput\AppData\Local\Temp\ipykernel_9996\319969289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Date,direction
0,2018-01-04,0.0
1,2018-01-05,1.0
2,2018-01-09,0.0
3,2018-01-10,0.0
4,2018-01-11,1.0
...,...,...
996,2021-12-24,1.0
997,2021-12-27,0.0
998,2021-12-28,0.0
999,2021-12-29,0.0


In [45]:
fig = go.Figure([go.Scatter(x=df["Date"], y=df[data_column])])
fig.show()

In [46]:
df = df.rename(columns={"Date": "ds", data_column: "y"})

train = df.iloc[:-200]
test = df.iloc[-200:]

model = Prophet()
model.fit(train)

23:05:55 - cmdstanpy - INFO - Chain [1] start processing
23:05:55 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
test

In [47]:
future = model.make_future_dataframe(periods=200)
forecast = model.predict(future)
preds = forecast[-200:]["yhat"]

In [48]:
preds = forecast["yhat"] > 0.5
preds = preds.astype(int)
preds = preds[-200:]

In [49]:
test["y"].astype(int)

801     0
802     0
803     1
804     0
805     0
       ..
996     1
997     0
998     0
999     0
1000    0
Name: y, Length: 200, dtype: int32

In [50]:
from sklearn.metrics import f1_score

f1_score(test["y"].astype(int), preds)

0.6220472440944882

In [54]:
class Forecaster:
    def __init__(self):
        self.forecast = None
        self.model = Prophet()

    def fit(self, data):
        self.model.fit(data)

    def predict(self, forecast_period_days=365):
        # call after self.fit()
        future = self.model.make_future_dataframe(periods=forecast_period_days)
        forecast = self.model.predict(future)
        self.forecast = forecast
        return forecast

    def export(self, export_filename="forecast.xlsx"):
        # call after self.predict()
        # Выгружаем прогноз в эксель. Спрогнозированное значение лежит в столбце yhat
        self.forecast.to_excel(
            export_filename, sheet_name="Data", index=False, encoding="cp1251"
        )

In [60]:
price_forecaster = Forecaster()

price_df = pd.read_csv("data.csv")
price_df = price_df.rename(
    columns={"дата": "Date", "направление": "direction", "выход": "value"}
)
price_df = price_df[["Date", "value"]]

price_df["Date"] = pd.to_datetime(price_df["Date"], dayfirst=True)
price_df = price_df.iloc[::-1].reset_index(drop=True)

price_df["value"] = price_df["value"].str.replace(",", ".").astype(float)

price_df = price_df.rename(columns={"Date": "ds", "value": "y"})

price_forecaster.fit(price_df)
preds = price_forecaster.predict()

# submit
submit_df = pd.read_csv("test.csv")
submit_df = submit_df.rename(
    columns={"дата": "Date", "направление": "direction", "выход": "value"}
)
submit_df = submit_df[["Date", "value"]]

submit_df["Date"] = pd.to_datetime(submit_df["Date"], dayfirst=True)
result = submit_df.merge(preds, left_on="Date", right_on="ds")["yhat"].tolist()

23:09:33 - cmdstanpy - INFO - Chain [1] start processing
23:09:33 - cmdstanpy - INFO - Chain [1] done processing


In [7]:
init_notebook_mode(connected=True)

iplot(
    [
        go.Scatter(x=price_df["ds"], y=price_df["y"], name="fact"),
        go.Scatter(x=forecast["ds"], y=forecast["yhat"], name="prediction"),
        go.Scatter(
            x=forecast["ds"],
            y=forecast["yhat_upper"],
            fill="tonexty",
            mode="none",
            name="upper",
        ),
        go.Scatter(
            x=forecast["ds"],
            y=forecast["yhat_lower"],
            fill="tonexty",
            mode="none",
            name="lower",
        ),
        go.Scatter(x=forecast["ds"], y=forecast["trend"], name="trend"),
    ]
)

In [75]:
direction_forecaster = Forecaster()

direction_df = pd.read_csv("data.csv")
direction_df = direction_df.rename(
    columns={"дата": "Date", "направление": "direction", "выход": "value"}
)
direction_df = direction_df[["Date", "direction"]]

direction_df["Date"] = pd.to_datetime(direction_df["Date"], dayfirst=True)
direction_df = direction_df.iloc[::-1].reset_index(drop=True)

direction_df = direction_df.replace({"direction": {"ш": 0, "л": 1}})
direction_df["direction"] = direction_df["direction"].astype(int)

direction_df = direction_df.rename(columns={"Date": "ds", "direction": "y"})

direction_forecaster.fit(direction_df.iloc[:-200])
preds = direction_forecaster.predict(200)
preds["yhat"] = preds["yhat"] > 0.5
preds["yhat"] = preds["yhat"].astype(int)

# submit
submit_df = pd.read_csv("test.csv")
submit_df = submit_df.rename(
    columns={"дата": "Date", "направление": "direction", "выход": "value"}
)
submit_df = submit_df[["Date", "direction"]]

submit_df["Date"] = pd.to_datetime(submit_df["Date"], dayfirst=True)
result = submit_df.merge(preds, left_on="Date", right_on="ds")["yhat"].tolist()

C:\Users\laput\AppData\Local\Temp\ipykernel_9996\2130307473.py:12: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

23:19:12 - cmdstanpy - INFO - Chain [1] start processing
23:19:12 - cmdstanpy - INFO - Chain [1] done processing


In [76]:
from sklearn.metrics import f1_score

f1_score(test["y"].astype(int), preds["yhat"][-200:])

0.6220472440944882